<a href="https://colab.research.google.com/github/waishun78/hungry-rag/blob/main/notebooks/AnswerGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load model with Float16 or Int8




In [ ]:
!pip install accelerate
!pip install bitsandbytes
# !pip install transformers
!pip install SentencePiece
!pip install evaluate
!pip install bert_score
!pip install googlemaps

In [ ]:
!nvidia-smi

Thu Nov  2 16:07:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Larger LLM Inferences

In [ ]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import LlamaTokenizer


load_in_8bit = True
model_name_l = "lmsys/vicuna-7b-v1.5"

model_l = AutoModelForCausalLM.from_pretrained(
    model_name_l,
    torch_dtype=torch.float16,
    load_in_8bit=load_in_8bit,
    device_map="auto"
)
tokenizer_l = LlamaTokenizer.from_pretrained(model_name_l)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
!pip install llama_index

In [ ]:
from llama_index import ServiceContext, LLMPredictor, HuggingFaceEmbedding, PromptHelper
from llama_index.llms import HuggingFaceLLM
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser

llm = HuggingFaceLLM(
      generate_kwargs={"temperature": 0.7, "do_sample": False},
    tokenizer_name="lmsys/vicuna-7b-v1.5",
    model_name="lmsys/vicuna-7b-v1.5",
)
embed_model = HuggingFaceEmbedding(


)
node_parser = SimpleNodeParser.from_defaults(
  text_splitter=TokenTextSplitter(chunk_size=1024, chunk_overlap=20)
)
prompt_helper = PromptHelper(
  context_window=4096,
  num_output=256,
  chunk_overlap_ratio=0.1,
  chunk_size_limit=None
)

service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model=embed_model,
  node_parser=node_parser,
  prompt_helper=prompt_helper
)

In [ ]:
# questions = [
#     "Who is the first prime minister of Singapore?",
#     "Who wrote the book named 'A Brief History of Time'? ",
#     "Who developed the general relativity between 1907 and 1915?",
#     "Who is the director of the film/TV series 'John Wick'?"
# ]
questions = [
    "Where to get budget-friendly Mexican food in Orchard Road with vegan options?",
    "Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere?",
    # "Where to get halal food in Geylang Serai which is open for breakfast and has a casual setting?",
    "I want to celebrate New Year's Eve. Where would you recommend I go?",
    "I'm looking for a casual restaurant for a date night. What are some suggestions?",
    """Here is a list of things I want to get:
    Matcha
    Chinese Noodles
    Ice cream dessert
    Can you provide me with suggestions?
    """
    # "I'm looking for a unique restaurant for a date night. What are some suggestions?",
    # "I'm looking for a restaurant with a view for a date night. What are some suggestions?"
]

ground_truth = [
    "Lee Kuan Yew",
    "Stephen Hawking",
    "Albert Einstein",
    "Chad Stahelski"
]

In [ ]:
def response_gen(input_ids):
  temperature=0.7
  with torch.no_grad():
      generation_output = model_l.generate(
          input_ids=input_ids,
          temperature=temperature,
          top_p = 1.0,
          do_sample=True,
          return_dict_in_generate=True,
          max_new_tokens=200,
      )
  s = generation_output.sequences[0][len(input_ids[0]):]
  output = tokenizer_l.decode(s)
  return output

In [ ]:
from transformers import GenerationConfig

predictions = []
for question in questions:
  print("Questions:", question)
  prompt = f"USER: {question}\n, BOT: Here are some follow up questions I need to better answer your query:"
  # input = tokenizer_l(question, return_tensors='pt')
  input = tokenizer_l(prompt, return_tensors='pt')
  input_ids = input["input_ids"].to("cuda")

  output = response_gen(input_ids)
  print("Prediction:", output)
  predictions.append(output)

Questions: Where to get budget-friendly Mexican food in Orchard Road with vegan options?
Prediction: 

1. How far are you willing to travel from Orchard Road to find a budget-friendly Mexican restaurant with vegan options?
2. Are you open to trying street food options or restaurants with a more casual dining atmosphere?
3. Do you have any specific dietary restrictions or allergies that we should be aware of?
4. Do you prefer traditional Mexican dishes or are you open to trying fusion dishes?

Please let me know the answers to these questions so I can better assist you in finding the best budget-friendly Mexican restaurant with vegan options on Orchard Road.</s>
Questions: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere?
Prediction: 

1. What type of Japanese cuisine are you looking for (e.g., sushi, teppanyaki, tempura, etc.)?
2. Are you looking for a specific type of dish or preference (e.g., seafood, meat, vegetarian, etc.)?
3. Do you 

In [ ]:
answers = [
    "1. Taco, 2. NIL, 3.Sit down"
  , "1. Sushi, Soup, 2. 30SGD, 3. NIL, 4. NIL, 5. NIL"
  , "1. Orchard, 2. Around 50$SGD, 3. Chill night, 4. NIL"
  , "1. Tanjong Pagar, 2. 15-20SGD, 5. No"
  , "1. Unsweetened, 3. Chocolate"]

search_keywords = []

for question, prediction, answer in zip(questions, predictions, answers):
  prompt2 = f"USER: {question}\n BOT:{prediction}\n USER: {answer} \n SEARCH QUERIES(BULLETS):"
  print("\n#################################\n")
  print("Prompt Combined.\n", prompt2)
  # print("User Input:", user_input)

  # input = tokenizer_l(question, return_tensors='pt')
  input = tokenizer_l(prompt2, return_tensors='pt')
  input_ids = input["input_ids"].to("cuda")

  output = response_gen(input_ids)
  print("Prediction:", output)
  print("\n#################################\n")
  search_keywords.append(output)


#################################

Prompt Combined.
 USER: Where to get budget-friendly Mexican food in Orchard Road with vegan options?
 BOT:

1. How far are you willing to travel from Orchard Road to find a budget-friendly Mexican restaurant with vegan options?
2. Are you open to trying street food options or restaurants with a more casual dining atmosphere?
3. Do you have any specific dietary restrictions or allergies that we should be aware of?
4. Do you prefer traditional Mexican dishes or are you open to trying fusion dishes?

Please let me know the answers to these questions so I can better assist you in finding the best budget-friendly Mexican restaurant with vegan options on Orchard Road.</s>
 USER: 1. Taco, 2. NIL, 3.Sit down 
 SEARCH QUERIES(BULLETS):
Prediction: 

* Mexican food on Orchard Road
* Budget-friendly Mexican restaurants
* Vegan options in Mexican cuisine
* Traditional Mexican dishes
* Mexican fusion cuisine
* Orchard Road Mexican restaurants
* Mexican street fo

In [ ]:
for search_keyword in search_keywords:
  print(repr(search_keywords))


['\n\n* Mexican food on Orchard Road\n* Budget-friendly Mexican restaurants\n* Vegan options in Mexican cuisine\n* Traditional Mexican dishes\n* Mexican fusion cuisine\n* Orchard Road Mexican restaurants\n* Mexican street food options\n* Casual dining atmosphere in Mexican restaurants\n* Vegan-friendly restaurants in Orchard Road</s>', '\n\n* Fine-dining sushi restaurant in Marina Bay Sands\n* Romantic sushi restaurant in Marina Bay Sands\n* Sushi restaurant with soup in Marina Bay Sands\n* Affordable sushi restaurant in Marina Bay Sands\n* Sushi restaurant with no dietary restrictions in Marina Bay Sands</s>', "\n\n1. Celebrate New Year's Eve in Singapore\n2. Affordable New Year's Eve ideas Singapore\n3. Chill New Year's Eve parties Singapore\n4. New Year's Eve events Singapore\n5. Things to do for New Year's Eve Singapore\n6. Best places to celebrate New Year's Eve Singapore\n7. New Year's Eve celebration ideas Singapore\n8. Cheap New Year's Eve activities Singapore\n9. New Year's Ev

In [2]:
questions = [
    "Where to get dim sum in Chinatown which is open late and has a view of the river?"
  , "Where to get Indian food in Singapore that is near the Esplanade and suitable for a group of 10 people?"
  ]

contexts = [
    """
    Victor's Kitchen
          Rating - 2.2
          Location - Chinatown Ave 43
          Opening - 7:30 to 15:00
          Available as - Dine-in, Takeaway, Delivery
          Reviews - "Ok food, not the best"
          Price: $$

    Yum Cha Chinatown
        Rating - 4.2
        Location - Zhongguocheng Street 33
        Opening - 09:30 to 22:00
        Available as - Dine-in, Takeaway, Delivery
        Reviews - "Enjoyable atmoshphere"
        Price: $$

    香港旺角點心 Hong Kong Mong Kok Tim Sum
        Rating - 5.0
        Location - Xinhao 22 Str. Ave
        Opening - 11:30 to 23:00
        Available as - Dine-in, Takeaway, Delivery
        Reviews - "Really good food", "Nice view of river"
        Price: $$
      """,

    """
      Shahi Maharani North Indian Restaurant
          Rating - 4.2
          Location - 6 Raffles Blvd, #04-600 PARKROYAL COLLECTION Marina Bay
          Opening - 11:30 to 19:00
          Available as - Dine-in, Takeaway, Delivery
          Reviews - "Fit for a big crowd"
          Price: $$
      Briyani Hut
          Rating - 4.2
          Location - 120 Raffles Str
          Opening - 11:30 to 19:00
          Available as - Dine-in, Takeaway, Delivery
          Reviews - "A bit squeezy but lovely food"
          Price: $$
      RedChillies Indian Cuisine
          Rating - 4.2
          Location - Park Ave 443
          Opening - 11:30 to 19:00
          Available as - Dine-in, Takeaway, Delivery
          Reviews - "Nice restaurant with cool lighting", "Nice boss"
          Price: $
    """
]

In [10]:
#Implementation of Google API '
import googlemaps

# from API_key import get_my_key #Imports personal API key

# # Define the API Key.
# API_KEY = get_my_key()

# # Define the Client
# gmaps = googlemaps.Client(key = API_KEY)

gmaps = googlemaps.Client(key = "REDACTED")
contexts_gplaces = []
for q in questions:
  places_result  = gmaps.places(q,region = "SG" ) #Demo query
  question_results = ""

  for i in range(3): #Top 3 results
      name = places_result['results'][i].get('name')
      rating = str(places_result['results'][i].get('rating'))
      price_level = places_result['results'][i].get('price_level')
      types = places_result['results'][i].get('types')
      open_now = "Yes" if places_result['results'][i]['opening_hours'].get('open_now') else "No"

      search_result = (f" Option {i+1}: {name}, Rating: {rating}, Price Level: {price_level} / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive), Shop Properties:{types}, Open Now:{open_now} ")
      question_results += search_result
  contexts_gplaces.append(question_results)

In [11]:
for i in contexts_gplaces:
  print(i)

 Option 1: Victor's Kitchen, Rating: 3.8, Price Level: 1 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive), Shop Properties:['meal_delivery', 'bakery', 'store', 'restaurant', 'point_of_interest', 'food', 'establishment'], Open Now:Yes  Option 2: Yum Cha Chinatown, Rating: 4, Price Level: 2 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive), Shop Properties:['restaurant', 'point_of_interest', 'food', 'establishment'], Open Now:No  Option 3: 香港旺角點心 Hong Kong Mong Kok Tim Sum, Rating: 4.2, Price Level: None / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive), Shop Properties:['restaurant', 'point_of_interest', 'food', 'establishment'], Open Now:No 
 Option 1: Rang Mahal - Fine Dining Indian Restaurant, Rating: 4.1, Price Level: 4 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive), Shop Properties:['restaurant', 'point_of_interest', 'food', 'establishment'], Open Now:No  Option 2: Punjab Gri

In [ ]:
generated_outputs = []

for question, context in zip(questions, contexts):
  prompt2 = f'''
    EXAMPLE OF OUTPUT:
    Query: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at 20:00?
    Context:
      [
        1:"Sushi Jiro At Parkroyal Marina Bay
          Rating - 4.2
          Location - 6 Raffles Blvd, #04-600 PARKROYAL COLLECTION Marina Bay
          Opening - 11:30 to 19:00
          Available as - Dine-in, Takeaway, Delivery
          Reviews - "Decent food", "More casual place", "Good family restaurant"
          Price: $$$
          ",
        2: "KOMA Singapore
          Rating - 4.2
          Location - 6 Raffles Blvd, #04-600 PARKROYAL COLLECTION Marina Bay
          Opening - 11:30 to 22:00
          Available as - Dine-in, Takeaway, Delivery
          Reviews - "Japanese eatery with striking decor.", "Great for a date night",
          Price: $$"

        3: "Sen of Japan
          Rating - 4.2
          Location - 6 Raffles Blvd, #04-600 PARKROYAL COLLECTION Marina Bay
          Opening - 11:30 to 23:00
          Available as - Dine-in, Takeaway, Delivery
          Reviews - "Casual but also could be a formal date.", "Enjoyable meal with good food",
          Price: $$"
      ]
    Answer:
      According to my research,
      1. KOMA Singapore[2] is the best option as it has a 4.2 rating [Rating] and is open until 22:00 [Opening]. The reviews also align with your preferences [Reviews] being a "Great for a date night".
      2. Sen of Japan[3] is a good alternative at 4.2 rating [Rating] and is open until 22:00 [Opening]. The reviews are relative relevant as it is both "casual but also could be a formal date." [Reviews].

    Query: {question}
    Context:{context},
    Answer:
    '''
  print("\n#################################\n")
  print("Prompt Combined.\n", prompt2)
  # print("User Input:", user_input)

  # input = tokenizer_l(question, return_tensors='pt')
  input = tokenizer_l(prompt2, return_tensors='pt')
  input_ids = input["input_ids"].to("cuda")

  output = response_gen(input_ids)
  print("Prediction: \n", output)
  print("\n#################################\n")
  generated_outputs.append(output)


#################################

Prompt Combined.
 
    EXAMPLE OF OUTPUT:
    Query: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at 20:00?
    Context:
      [
        1:"Sushi Jiro At Parkroyal Marina Bay
          Rating - 4.2
          Location - 6 Raffles Blvd, #04-600 PARKROYAL COLLECTION Marina Bay
          Opening - 11:30 to 19:00
          Available as - Dine-in, Takeaway, Delivery
          Reviews - "Decent food", "More casual place", "Good family restaurant"
          Price: $$$
          ",
        2: "KOMA Singapore
          Rating - 4.2
          Location - 6 Raffles Blvd, #04-600 PARKROYAL COLLECTION Marina Bay
          Opening - 11:30 to 22:00
          Available as - Dine-in, Takeaway, Delivery
          Reviews - "Japanese eatery with striking decor.", "Great for a date night",
          Price: $$"

        3: "Sen of Japan
          Rating - 4.2
          Location - 6 Raffles Blvd, #04-600 PARKROYAL COLLECTI

# BLEU Score

In [ ]:
import evaluate
import numpy as np
bleu = evaluate.load("bleu")
bleu_score = bleu.compute(predictions=predictions, references=ground_truth)
print(f"BLEU Score: {bleu_score}")

BLEU Score: {'bleu': 0.0, 'precisions': [0.0423728813559322, 0.02631578947368421, 0.00909090909090909, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 13.11111111111111, 'translation_length': 118, 'reference_length': 9}


# BERTScore

In [ ]:
bertscore = evaluate.load("bertscore")
bert_score = bertscore.compute(predictions=predictions, references=ground_truth, lang="en")
print(f"BERTScore: {bert_score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: {'precision': [0.7950500249862671, 0.8199793100357056, 0.853007435798645, 0.7685672044754028], 'recall': [0.8957263231277466, 0.9084771275520325, 0.797988772392273, 0.6577140092849731], 'f1': [0.8423908352851868, 0.8619626760482788, 0.8245813846588135, 0.7088327407836914], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.34.1)'}


# Larger LLM Verification

# Wiki-data Assistance


In [ ]:
wiki_data = [
    "After leading the PAP to victory in seven elections, Lee stepped down on 28 November 1990, handing over the prime ministership to Goh Chok Tong. By that time, he had become the world's longest-serving prime minister.",
    "Stephen Hawking achieved commercial success with several works of popular science in which he discussed his theories and cosmology in general. His book A Brief History of Time appeared on the Sunday Times bestseller list for a record-breaking 237 weeks. ",
    "Albert Einstein developed the General relativity (GR), which is a theory of gravitation between 1907 and 1915. According to it, the observed gravitational attraction between masses results from the warping of spacetime by those masses.",
    "Based on the context, John Wick is a 2014 American action thriller film directed by Chad Stahelski and written by Derek Kolstad. Keanu Reeves stars as John Wick, a legendary hitman who comes out of retirement to seek revenge against the men who killed his puppy, a final gift from his recently deceased wife. The film also stars Michael Nyqvist, Alfie Allen, Adrianne Palicki, Bridget Moynahan, Dean Winters, Ian McShane, John Leguizamo, and Willem Dafoe."
]

In [ ]:
from transformers import GenerationConfig

verifications = []
for i in range(len(questions)):
  question = questions[i]
  answer = ground_truth[i]
  prompt = f"Context: {wiki_data[i]}\n\n Question: {question}\n\n Answer: the answer is {answer} \n\n Based on the following context, is the answer correct to the question? Please answer YES or NO"
  input = tokenizer_l(prompt, return_tensors='pt')
  input_ids = input["input_ids"].to("cuda")

  # with torch.cuda.amp.autocast():
  #   output_tokens = model.generate(**input, max_new_tokens=50)

  # print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

  output = response_gen(input_ids)
  print("Question:", question)
  print("Ground_truth:", answer)
  print("Larger LLM Verification:", output)
  verifications.append(output)

Question: Who is the first prime minister of Singapore?
Ground_truth: Lee Kuan Yew
Larger LLM Verification: .

 YES.</s>
Question: Who wrote the book named 'A Brief History of Time'? 
Ground_truth: Stephen Hawking
Larger LLM Verification: .

 YES</s>
Question: Who developed the general relativity between 1907 and 1915?
Ground_truth: Albert Einstein
Larger LLM Verification: .
 YES</s>
Question: Who is the director of the film/TV series 'John Wick'?
Ground_truth: Chad Stahelski
Larger LLM Verification: .

NO.</s>


In [ ]:
from transformers import GenerationConfig

for i in range(len(questions)):
  question = questions[i]
  print("Questions:", question)
  prompt = f"Please answer the question based on the context.\n Context: {wiki_data[i]}\n Question: {question}"
  input = tokenizer_l(prompt, return_tensors='pt')
  input_ids = input["input_ids"].to("cuda")

  # with torch.cuda.amp.autocast():
  #   output_tokens = model.generate(**input, max_new_tokens=50)

  # print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

  output = response_gen(input_ids)
  print("Prediction:", output)

Questions: Who is the first prime minister of Singapore?
Prediction: 
 Answer: Lee Kuan Yew is the first prime minister of Singapore.</s>
Questions: Who wrote the book named 'A Brief History of Time'? 
Prediction: 
 Answer: Stephen Hawking wrote the book named 'A Brief History of Time'.</s>
Questions: Who developed the general relativity between 1907 and 1915?
Prediction: 
 Answer: Albert Einstein developed the General relativity between 1907 and 1915.</s>
Questions: Who is the director of the film/TV series 'John Wick'?
Prediction: 
 Answer: The director of the film/TV series 'John Wick' is Chad Stahelski.</s>


In [ ]:
example_1 = "Who developed the general relativity between 1920 and 1935?"
example_1_withcontext = """Please answer the question based on the context.
Context:
1931 – Subrahmanyan Chandrasekhar studies the stability of white dwarfs.
1931 – Georges Lemaître and Arthur Eddington predict the expansion of the Universe.
1931 – Albert Einstein introduces his cosmological constant.
1932 – Albert Einstein and Willem de Sitter propose the Einstein-de Sitter cosmological model.
1934 – Walter Baade and Fritz Zwicky predict the existence of neutron stars.[71] Although their details are wrong, their basic idea is now accepted.
1935 – Albert Einstein and Nathan Rosen derive the Einstein-Rosen bridge, the first wormhole solution.
1936 – Albert Einstein predicts that a gravitational lens brightens the light coming from a distant object to the observer.
1937 – Fritz Zwicky states that galaxies could act as gravitational lenses.
1937 – Albert Einstein and Nathan Rosen obtain the Einstein-Rosen metric, the first exact solution describing gravitational waves.
1938 – Albert Einstein, Leopold Infeld, and Banesh Hoffmann obtain the Einstein-Infeld-Hoffmann equations of motion.
1939 – Hans Bethe shows that nuclear fusion is responsible for energy production inside stars, building upon the Kelvin–Helmholtz mechanism.
1939 – Richard Tolman solves the Einstein field equations in the case of a spherical fluid drop.
1939 – Robert Serber, George Volkoff, Richard Tolman, and J. Robert Oppenheimer study the stability of neutron stars, obtaining the Tolman–Oppenheimer–Volkoff limit.
1939 – J. Robert Oppenheimer and Hartland Snyder publish the Oppenheimer-Snyder model for the continued gravitational contraction of a star.

Question: Who developed the general relativity between 1920 and 1935?
"""

In [ ]:
input = tokenizer_l(example_1, return_tensors='pt')
input_ids = input["input_ids"].to("cuda")
output = response_gen(input_ids)
print("Prediction:", output)

Prediction: 
, it was Einstein's theory of general relativity. Einstein's theory of relativity is a fundamental theory in physics that describes the relationship between space and time. It is based on the idea that space and time are not separate entities, but are instead interconnected and relative to each other. The theory was developed over a period of several years, beginning in the 1920s and continuing through the 1930s. Einstein's work


In [ ]:
input = tokenizer_l(example_1_withcontext, return_tensors='pt')
input_ids = input["input_ids"].to("cuda")
output = response_gen(input_ids)
print("Prediction:", output)

Prediction: 
Answer: Albert Einstein developed the general theory of relativity between 1920 and 1935.</s>
